<img align="left" src = "images/linea.png" width=140 style="padding: 20px"> 
<img align="left" src = "images/rubin.png" width=180 style="padding: 30px"> 

# PZ Compute 
## Photo-zs for DP0.2 Objects catalog

Contact: [Julia Gschwend](mailto:julia@linea.org.br)

Last verified run: **10-Jul-2024**


# Step 1a - Make a random training set using RSP TAP Service 
**Contributors:** Andreia Dourado, Julia Gschwend, Yago Lopes, Bruno Moraes. 


This section was developed based on the example notebook by Melissa Graham example notebook [simple_pz_training_set.ipynb](https://github.com/rubin-dp0/delegate-contributions-dp02/blob/9a020e68d04884f47d0f358c46af961daa0d8318/photoz/Training_Set_Creation/simple_pz_training_set.ipynb) and [discussion in Rubin Community Forum](https://community.lsst.org/t/adql-question-selecting-a-random-subset-of-objects-in-dp0/8022). 

It is set up to be executed on the [Rubin Science Platform](https://data.lsst.cloud/) and requires data right to access LSST data. 


Import packages and define parameters 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tables_io
import random
import os
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery

from lsst.rsp import get_tap_service, retrieve_query
%matplotlib inline

### Random selection 

**Query `objectId` using TAP service** 

To select random galaxies from the whole catalog, the strategy adopted is to query only the column `objectId`, perform a random selection on the resulting array using Pandas, then query the training set columns from the database, only for the pre-selected objects. See discussion on the Community Forum [here](https://community.lsst.org/t/adql-question-selecting-a-random-subset-of-objects-in-dp0/8022). 

To reduce the size of the `objectId` array and avoid selecting useless objects, S/N and blending cuts are applied to this first query:  `i-band $S/N > 5` and `detect_isPrimary = 1`.  

```sql 
SELECT objectId  FROM dp02_dc2_catalogs.Object 
    WHERE detect_isPrimary=1 
    AND scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) < 0.2 
```

Select a subsample limited to 500k objects as example (one should query the entire object catalog to obtain a real random sample).  

In [ ]:
service = get_tap_service("tap")
query = "SELECT TOP 500000 objectId FROM dp02_dc2_catalogs.Object WHERE detect_isPrimary=1 AND scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) < 0.2" 

Synchronous query

In [ ]:
results = service.search(query).to_table().to_pandas()
results

Asynchronous query

In [ ]:
#job = service.submit_job(query)
#print('Job URL is', job.url)
#print('Job phase is', job.phase)

In [ ]:
#job.run()

In [ ]:
#print('Job phase is', job.phase)

In [ ]:
#%%time
#job.wait(phases=['COMPLETED', 'ERROR'])
#print('Job phase is', job.phase)

In [ ]:
#job.raise_if_error()

In [ ]:
#results = job.fetch_result().to_table().to_pandas()

Clean up

In [ ]:
#job.delete()

In [ ]:
#dp02_size = 278_318_455
dp02_size = 500_000
assert len(results) == dp02_size

Store `objectId` array in a CSV file 

In [ ]:
results.to_csv('objectIds_500k_galaxies_sn_max_5.csv')
#async_results.to_csv('objectIds_all_galaxies_sn_max_5.csv')

**Select a random fraction of objects with Pandas**

In [ ]:
training_set_size = 100_000 # approximate desired training set size
random_fraction = training_set_size/dp02_size 
print(f"Training set is a random fraction of: {random_fraction*100:.2f} percent") 

In [ ]:
rsp_user = 'gschwend' # replace with RSP username 
#filename = f'/home/{rsp_user}/WORK/pz-compute/docs/notebooks/objectIds_all_galaxies_sn_max_5.csv'
filename = f'/home/{rsp_user}/WORK/pz-compute/docs/notebooks/objectIds_500k_galaxies_sn_max_5.csv'
random.seed(4321) # choose a seed if you want to reproduce the same random selection later 
random_ids = pd.read_csv(filename, 
           skiprows=lambda x: x > 0 and random.random() > random_fraction) 

In [ ]:
random_ids

### Query training set columns 

To query the training set columns, i.e., observables and true redshifts, it is necessary to join the list of random `objectId's` previously selected with both `Object` and `TruthSummary` tables. See the complete list of columns present in each table on [DM's DP0 Schema browser page](https://dm.lsst.org/sdm_schemas/browser/dp02.html).  

To avoid reaching memory limits on RSP, split the data to obtain ~20,000 objects in each query.  

In [ ]:
query_limit = 20_000
n_times = int(training_set_size/query_limit)
print(f"The query to retrieve the training set columns must be done in {n_times} parts, then the results will be concatenated.")  

Prepare the set of files with random selection for each query run

In [ ]:
for i in range(n_times):
    start_idx = i * query_limit
    end_idx = (i + 1) * query_limit
    part_random_ids = random_ids.iloc[start_idx:end_idx]
    part_random_ids.to_csv(f'random_objectId_part_{i+1}.csv', index=False)

Execute the queries in sequence (this cell takes ~25 minutes to run). 

In [ ]:
%%time
for i in range(n_times):
    # Writting the objectIds in the query format
    random_ids = pd.read_csv(f'random_objectId_part_{i+1}.csv')['objectId']
    ids = np.squeeze(random_ids.values)
    str_ids = str(tuple(ids))
    
    query = '\n'.join(['SELECT obj.objectId, obj.coord_ra as ra, obj.coord_dec as dec, ts.redshift, ',  
                       'scisql_nanojanskyToAbMag(obj.u_cModelFlux) as mag_u, ', 
                       'scisql_nanojanskyToAbMag(obj.g_cModelFlux) as mag_g, ', 
                       'scisql_nanojanskyToAbMag(obj.r_cModelFlux) as mag_r, ', 
                       'scisql_nanojanskyToAbMag(obj.i_cModelFlux) as mag_i, ', 
                       'scisql_nanojanskyToAbMag(obj.z_cModelFlux) as mag_z, ', 
                       'scisql_nanojanskyToAbMag(obj.y_cModelFlux) as mag_y, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.u_calibFlux, obj.u_calibFluxErr) as magerr_u, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.g_calibFlux, obj.g_calibFluxErr) as magerr_g, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.r_calibFlux, obj.r_calibFluxErr) as magerr_r, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.i_calibFlux, obj.i_calibFluxErr) as magerr_i, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.z_calibFlux, obj.z_calibFluxErr) as magerr_z, ', 
                       'scisql_nanojanskyToAbMagSigma(obj.y_calibFlux, obj.y_calibFluxErr) as magerr_y ',
            'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
            'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
            'ON mt.id_truth_type=ts.id_truth_type ',
            'JOIN dp02_dc2_catalogs.Object AS obj ',
            'ON mt.match_objectId=obj.objectId ', 
            'WHERE obj.objectId IN'+str_ids,
            'AND ts.truth_type=1 ',
            'AND obj.detect_isPrimary=1'])
    
    results = service.search(query).to_table().to_pandas()
    results.to_csv(f"random_training_set_part_{i+1}.csv")

**Concatenate results and remove partial files**

In [ ]:
training_set = pd.DataFrame()
for i in range(n_times):
    df = pd.read_csv(f"random_training_set_part_{i+1}.csv")
    training_set = pd.concat([training_set, df], axis=0).reset_index(drop=True)
    #os.system(f"rm random_training_set_part_{i+1}.csv")
    #os.system(f"rm random_objectId_part_{i+1}.csv")

Check training set dataframe

In [ ]:
training_set.describe()

### Save training set file

**Data cleaning and extinction correction** 


Apply extinction correction on the magnitudes

See Sam Schmidt's example notebooks[Derive_A_EBV_coefficients.ipynb](https://github.com/LSSTDESC/DC2-analysis/blob/master/contributed/Derive_A_EBV_coefficients.ipynb) and [DC2_object_deredden.ipynb](https://github.com/LSSTDESC/DC2-analysis/blob/master/contributed/DC2_object_deredden.ipynb), and Jeff Carlin's notebook [Milky_Way_halo_density.ipynb](https://github.com/rubin-dp0/delegate-contributions-dp02/blob/main/MWhalo_density/Milky_Way_halo_density.ipynb) on [DP0.2 delegates's contributions repository](https://github.com/rubin-dp0/delegate-contributions-dp02) for reference. 

In [ ]:
# A_lamba/E(B-V) values for LSST filters 
band_a_ebv = np.array([4.81,3.64,2.70,2.06,1.58,1.31])

In [ ]:
# dustmaps expects coordinates in Astropy "SkyCoord" format 
coords = c = SkyCoord(training_set['ra'], training_set['dec'], unit = 'deg',frame='icrs')

In [ ]:
sfd = SFDQuery()
ebv = sfd(coords)
#training.add_column(ebvvec, name='ebv')

In [ ]:
ebv

In [ ]:
training_set['mag_u'] = training_set['mag_u'] - ebv*band_a_ebv[0]
training_set['mag_g'] = training_set['mag_g'] - ebv*band_a_ebv[1]
training_set['mag_r'] = training_set['mag_r'] - ebv*band_a_ebv[2]
training_set['mag_i'] = training_set['mag_i'] - ebv*band_a_ebv[3]
training_set['mag_z'] = training_set['mag_z'] - ebv*band_a_ebv[4]
training_set['mag_y'] = training_set['mag_y'] - ebv*band_a_ebv[4]

In [ ]:
ug0 = training_set['mag_u'] - training_set['mag_g']
gr0 = training_set['mag_g'] - training_set['mag_r']
ri0 = training_set['mag_r'] - training_set['mag_i']
iz0 = training_set['mag_i'] - training_set['mag_z']

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(2,3,1)
plt.hist(training_set['mag_u'], bins=50)
plt.subplot(2,3,2)
plt.hist(training_set['mag_g'], bins=50)
plt.subplot(2,3,3)
plt.hist(training_set['mag_r'], bins=50)
plt.subplot(2,3,4)
plt.hist(training_set['mag_i'], bins=50)
plt.subplot(2,3,5)
plt.hist(training_set['mag_z'], bins=50)
plt.subplot(2,3,6)
plt.hist(training_set['mag_y'], bins=50)
plt.subplots_adjust()

In [ ]:
plt.scatter(training_set.z, training_set.mag_i, s=10)
plt.xlabel("true redshift",fontsize=14)
plt.ylabel("mag i",fontsize=14)

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(1,3,1)
plt.scatter(training_set.mag_g, ug0, s=10)
plt.xlabel("g",fontsize=14)
plt.ylabel("u-g",fontsize=14)
plt.subplot(1,3,2)
plt.scatter(training_set.mag_r, gr0, s=10)
plt.xlabel("r",fontsize=14)
plt.ylabel("g-r",fontsize=14)
plt.subplot(1,3,3)
plt.scatter(training_set.mag_i,ri0,s=10)
plt.xlabel("i",fontsize=14)
plt.ylabel("r-i",fontsize=14)
plt.subplots_adjust()

In [ ]:
plt.figure(figsize=(12,3))
plt.subplot(1,3,1)
plt.scatter(ug0,gr0,s=10)
plt.xlabel("u-g",fontsize=14)
plt.ylabel("g-r",fontsize=14)
plt.subplot(1,3,2)
plt.scatter(gr0,ri0,s=10)
plt.xlabel("g-r",fontsize=14)
plt.ylabel("r-i",fontsize=14)
plt.subplot(1,3,3)
plt.scatter(ri0,iz0,s=10)
plt.xlabel("r-i",fontsize=14)
plt.ylabel("i-z",fontsize=14)
plt.subplots_adjust()

Remove magnitude and color extreme outliers 

In [ ]:
training_set.query('(mag_g - mag_r) < 8 & (mag_r - mag_i) < 6', inplace=True) 

Save training set in various file formats 

In [ ]:
training_set.to_csv('dp02_random_training_set.csv') 
training_set.to_hdf('dp02_random_training_set.hdf5', key='photometry') 
training_set.to_parquet('dp02_random_training_set.pq') 

### Final sample characterization 


In [ ]:
training_set = tables_io.read('dp02_random_training_set.pq')
training_set

On-sky spatial distribuition

In [ ]:
plt.scatter(training_set['ra'],training_set['dec'],s=1)
plt.colorbar()

# Step 1b - Make a random training set using IDAC-BR file system   
**Contributors:** Julia Gschwend, Luigi Silva, Heloisa Mengisztki, Cristiano Singulani.  


This section aims to reproduce the same procedure as section 1a, but retrieving data from the DP0.2 object catalog stored in parquet files on the Brazilian IDAC file system. 

It is set up to be executed on LIneA's JuputerHub or OnDemand services, and also requires data right to access LSST data. 